<a href="https://colab.research.google.com/github/AbdulWahabRaza123/NLP/blob/main/Word2VecNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#gensim is best NLP library which work with python-levenshtein
import gensim
import pandas as pd
import os

In [14]:
!wget https://github.com/masta-g3/amazon_nlp/blob/master/Cell_Phones_and_Accessories_5.json.gz

--2022-12-17 13:44:29--  https://github.com/masta-g3/amazon_nlp/blob/master/Cell_Phones_and_Accessories_5.json.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Cell_Phones_and_Accessories_5.json.gz’

Cell_Phones_and_Acc     [ <=>                ] 135.98K   741KB/s    in 0.2s    

2022-12-17 13:44:29 (741 KB/s) - ‘Cell_Phones_and_Accessories_5.json.gz’ saved [139239]



In [35]:
df = pd.read_json('/content/drive/MyDrive/Datasets/Cell_Phones_and_Accessories_5.json',lines=True)

In [36]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [38]:
df.shape

(194439, 9)

In [39]:
#we are only intrested in the review text column
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [41]:
#Now we have to remove the waste things like semi colons, punctuation marks etc
#this we can do using gensim library
gensim.utils.simple_preprocess(df.reviewText[0])

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [42]:
#let's create pandas series of these reviews
review_text=df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [43]:
#lets building the model
model=gensim.models.Word2Vec( 
    window=10,
    min_count=2,
    workers=4
)

In [44]:
model.build_vocab(review_text,progress_per=1000)

In [45]:
model.epochs

5

In [47]:
model.corpus_count

194439

In [48]:
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)

(61505913, 83868975)

In [49]:
model.save("./word2vec-amazon-cell-assessories-reviews-short.model")

In [50]:
#I saved model to use it for later
#now let's test our model
model.wv.most_similar("bad")

[('terrible', 0.6880035400390625),
 ('shabby', 0.6679180264472961),
 ('horrible', 0.6204333305358887),
 ('good', 0.5919754505157471),
 ('legit', 0.5600746273994446),
 ('awful', 0.5519806146621704),
 ('disappointing', 0.5395427942276001),
 ('cheap', 0.5331185460090637),
 ('okay', 0.5244956612586975),
 ('poor', 0.5234232544898987)]

In [52]:
#let's test the similarities between two words
model.wv.similarity(w1="cheap",w2="inexpensive")


0.51929677

In [53]:
model.wv.similarity(w1="great",w2="good")

0.7815587

In [54]:
model.wv.similarity(w1="great",w2="product")

-0.039412457